In [24]:
# import csv
from operator import itemgetter
import numpy as np
import pandas as pd
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'


# from datetime import datetime

file5 = open('201811291521.txt', 'r')
dataset = file5.read()
print('data read')
dataset = dataset.split('\n')
data = []
for i in range(0, len(dataset)):
    data.append(dataset[i].split('|'))
data = sorted(data, key=itemgetter(0))
data.pop(0)
print('data sorted')

headings = ['stime', 'etime', 'sip', 'sport', 'sipint', 'mac', 'osname', 'osversion', 'fingerprint', 'dip', 'dport', 'dipint', 'dstmac', 'rosname', 'rosversion', 'rfingerprint', 'protocol', 'pkts', 'bytes', 'rpkts', 'rbytes', 'dur', 'iflags', 'riflags', 'uflags', 'ruflags', 'entropy', 'rentropy', 'tos', 'rtos', 'application', 'vlanint', 'domain', 'endreason', 'hash']
print('data : '+str(len(data)))
# for item in data:
#     item[0] = datetime.strptime(item[0], '%Y-%m-%d %H:%M:%S.%f')
#     item[1] = datetime.strptime(item[1], '%Y-%m-%d %H:%M:%S.%f')
#     print(item[0].strftime('%m/%d/%Y'))

data = np.array(data)
df = pd.DataFrame(data)
df.columns = headings
print('dataset created')
print(df.head())
edited_df = df.drop(['stime','etime','sipint','mac','osname','osversion','fingerprint','dipint','dstmac','rosname','rosversion','rfingerprint','iflags','riflags','uflags','ruflags','entropy','rentropy','tos','rtos','application','vlanint','domain','hash','pkts','bytes','rpkts','rbytes','dur','endreason'],axis=1)
print(edited_df.head())
# Get one hot encoding of columns B
one_hot = pd.get_dummies(edited_df['protocol'])
# Drop column B as it is now encoded
edited_df = edited_df.drop('protocol',axis = 1)
headers = []
for i in one_hot.columns:
    headers.append('protocol_' + i)
# Join the encoded df

one_hot.columns = headers
# edited_df = edited_df.join(one_hot)

def correct_ip(s):
    o = ''
    if '.' in s:
        for part in s.split('.'):
            part = part.zfill(3)
            o += part 
    else:
        o = o.zfill(12)
    o = o[:3] + '.' + o[3:]
    o = o[:7] + '.' + o[7:]
    o = o[:11] + '.' + o[11:]
    return o

def correct_port(s):
    return(s.zfill(5))
        


sip_headers = []
dip_headers = []

for i in range(4):
    sip_headers.append('sip_'+str(i))
    dip_headers.append('dip_'+str(i))

sip = []
for ip in edited_df['sip']:
    sip.append(map(int,correct_ip(ip).split('.')))
#     sip.append(correct_ip(ip).split('.'))

dip = []
for ip in edited_df['dip']:
    dip.append(map(int,correct_ip(ip).split('.')))
#     dip.append(correct_ip(ip).split('.'))


        
sport = []
for port in edited_df['sport']:
    sport.append(int(port))
    
dport = []
for port in edited_df['dport']:
    dport.append(int(port))
# print(len(sip[0]))
# print(len(dip[0]))
# print(len(dport[0]))
# print(len(sport[0]))
8
sip_df = pd.DataFrame(sip,columns=sip_headers)
dip_df = pd.DataFrame(dip,columns=dip_headers)
sport_df = pd.DataFrame(sport,columns=['sport'])
dport_df = pd.DataFrame(sport,columns=['dport'])


result = pd.concat([sip_df, dip_df, sport_df, dport_df, one_hot], axis=1, sort=False)
result.head()

# print(df['protocol'].nunique())

data read
data sorted
data : 213446
dataset created
                     stime                    etime             sip  sport  \
0  2017-05-09 10:36:34.876  2017-05-09 11:05:37.894  192.168.80.102   5353   
1  2017-05-09 10:38:11.077  2017-05-09 11:06:13.402  192.168.110.60  34056   
2  2017-05-09 10:38:11.077  2017-05-09 11:06:13.402  192.168.110.60  34056   
3  2017-05-09 10:38:11.077  2017-05-09 11:06:13.402  192.168.110.60  34056   
4  2017-05-09 10:38:11.078  2017-05-09 11:06:13.402  192.168.110.60  34056   

       sipint                 mac osname osversion fingerprint  \
0  3232256102  20:cf:30:8b:6f:17:                                
1  3232263740  00:26:18:f0:62:08:                                
2  3232263740  00:26:18:f0:62:08:                                
3  3232263740  00:26:18:f0:62:08:                                
4  3232263740  00:26:18:f0:62:08:                                

               dip     ...     ruflags entropy rentropy tos rtos application  \
0 

,sip_0,sip_1,sip_2,sip_3,dip_0,dip_1,dip_2,dip_3,sport,dport,protocol_1,protocol_139,protocol_17,protocol_2,protocol_41,protocol_47,protocol_6
0,192,168,80,102,224,0,0,251,5353,5353,0,0,1,0,0,0,0
1,192,168,110,60,171,122,234,240,34056,34056,0,0,1,0,0,0,0
2,192,168,110,60,182,132,115,102,34056,34056,0,0,1,0,0,0,0
3,192,168,110,60,182,40,209,121,34056,34056,0,0,1,0,0,0,0
4,192,168,110,60,124,232,60,77,34056,34056,0,0,1,0,0,0,0


In [91]:
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model
from keras import regularizers
from keras import optimizers
import matplotlib.pyplot as plt
import keras.backend as K
from keras import losses


def my_own_loss():
        def port_loss(y_true, y_pred):
            x = K.square(y_pred - y_true)
            return K.mean(K.min(x-1,0)*x/(x-1) + K.max(x-1,0)*(2*x -1)/2/x/(x-1),axis=-1)
        
        def loss(y_true, y_pred):
            # Reconstruction loss
#             K.print_tensor(y_true, message='y_true = ')
            print(K.get_value(y_pred))
            sip0_loss = 4 * K.mean(K.square(y_pred[0] - y_true[0]), axis=-1)
            sip1_loss = 3 * K.mean(K.square(y_pred[1] - y_true[1]), axis=-1)
            sip2_loss = 2 * K.mean(K.square(y_pred[2] - y_true[2]), axis=-1)
            sip3_loss = K.mean(K.square(y_pred[3] - y_true[3]), axis=-1)
            
            sip_loss = sip0_loss + sip1_loss + sip2_loss + sip3_loss 

            dip0_loss = 16 * K.mean(K.square(y_pred[4] - y_true[4]), axis=-1)
            dip1_loss = 9 * K.mean(K.square(y_pred[5] - y_true[5]), axis=-1)
            dip2_loss = 4 * K.mean(K.square(y_pred[6] - y_true[6]), axis=-1)
            dip3_loss = K.mean(K.square(y_pred[7] - y_true[7]), axis=-1)
            
            dip_loss = dip0_loss + dip1_loss + dip2_loss + dip3_loss 
            
            sport_loss = K.mean(K.square(y_pred[8] - y_true[8]), axis=-1)
#             sport_loss = K.mean(K.min(x-1,0)*x/(x-1) + K.max(x-1,0)*(2*x -1)/2/x/(x-1),axis=-1)
#             sport_loss = port_loss(y_true[8],y_pred[8])
            dport_loss = 4 * K.mean(K.square(y_pred[8] - y_true[8]), axis=-1)
#             protocol_loss = K.mean(K.square(y_true[10:17]-y_pred[10:17]),axis=-1)
            protocol_loss = K.categorical_crossentropy(y_true, y_pred)
            # Full loss
            
            model_loss = protocol_loss + sip_loss + dip_loss + sport_loss + dport_loss
            return model_loss

        return loss

# def custom_loss():

#     # Create a loss function that adds the MSE loss to the mean of all squared activations of a specific layer
#     def loss(y_true,y_pred):
#         return K.mean(K.square(y_pred - y_true), axis=-1)
#     return loss
    
x_train,x_test = train_test_split(result, test_size=0.2)
print(x_train.shape, x_test.shape)

input_d = Input(shape=(17,))
encoded = Dense(14, activation='relu')(input_d)
encoded = Dense(10, activation='relu')(encoded)
encoded = Dense(6, activation='relu')(encoded)
encoded = Dense(3, activation='relu')(encoded)

decoded = Dense(6, activation='relu')(encoded)
decoded = Dense(10, activation='relu')(decoded)
decoded = Dense(14, activation='relu')(decoded)
decoded = Dense(17, activation='sigmoid')(decoded)

autoencoder = Model(input_d, decoded)
# optimizer = optimizers.SGD(lr=0.1, momentum=0.0, decay=0.1)

autoencoder.compile(optimizer='RMSprop', loss=my_own_loss(),metrics=['accuracy'])

n_folds = 10
fold_len = int(len(x_train)/n_folds)
cross_validation_scores = []
for i in range(n_folds):
    print("Running Fold", i+1, "/", n_folds)
    fold_train = [k for k in range(0,i*fold_len)] + [k for k in range((i+1)*fold_len,len(x_train))]
    fold_test = [item for item in range(i*fold_len,(i+1)*fold_len)]
    test = x_train.iloc[fold_test]
    train = x_train.iloc[fold_train]
    history = autoencoder.fit(train, train,
            epochs=10,
            batch_size=256,
            shuffle=True,
            validation_data = (test,test))
    fold_validation_score = autoencoder.evaluate(test,test,verbose=0)
    print("fold validation score = ", fold_validation_score)
    cross_validation_scores.append(fold_validation_score[1] * 100)
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

print("%.2f%% (+/- %.2f%%)" % (np.mean(cross_validation_scores), np.std(cross_validation_scores)))
    
# autoencoder.fit(x_train, x_train,
#             epochs=100,
#             batch_size=256,
#             shuffle=True,
#             validation_split = 0.2)



(170756, 17) (42690, 17)


InvalidArgumentError: You must feed a value for placeholder tensor 'input_47' with dtype float and shape [?,17]
	 [[node input_47 (defined at /anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517)  = Placeholder[dtype=DT_FLOAT, shape=[?,17], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'input_47', defined at:
  File "/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/anaconda3/lib/python3.6/site-packages/tornado/platform/asyncio.py

In [82]:
test = autoencoder.evaluate(x=x_test,y=x_test, verbose=2,batch_size=256)    
# print(len(test))

In [69]:
list_of_losses = []
batch_list = []
for index, row in result.iterrows():
    input_row = np.array([result.iloc[index].values])
    batch_list.append(input_row)
    if len(batch_list) == 256:
        loss = autoencoder.evaluateb(atch_list,batch_list,verbose=2,batch_size=256)
#         if index%1000 == 0:
        print(index)
        batch_list = []
        list_of_losses.append((index,loss[1]))
print('list is ready to sort')


ValueError: Error when checking input: expected input_39 to have 2 dimensions, but got array with shape (256, 1, 17)

In [83]:
list_of_losses = []
print(my_own_loss())
for index, row in result.iterrows():
    input_row = np.array([result.iloc[index].values])
    y_pred = autoencoder.predict(input_row)
    print(y_pred)
    print(my_own_loss()(input_row,y_pred))
    if index%1000 == 0:
        print(index)
    list_of_losses.append((index,loss[1]))
print('list is ready to sort')

<function my_own_loss.<locals>.loss at 0x145d3bae8>
[[3.10109574e-02 2.14319751e-02 8.36747419e-03 1.25838527e-02
  2.97226273e-02 1.09252585e-02 4.16391855e-03 3.48312706e-02
  1.00000000e+00 1.00000000e+00 8.99728250e-22 9.37191514e-21
  4.08808555e-05 3.17785040e-21 2.23582881e-21 1.59932009e-21
  3.62923272e-21]]


IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
list_of_losses.sort(key=lambda x: x[1])
print(list_of_losses[:10])